In [15]:
import pandas as pd
import json

# Leer archivo Excel
df = pd.read_excel("revision_sponsors.xlsx")

# Columnas requeridas
required_columns = ['prompt', 'descripcion_video', 'respuesta_modelo']
if not all(column in df.columns for column in required_columns):
    raise ValueError(f"El archivo Excel debe contener las columnas: {required_columns}")

# Crear archivo JSONL (una línea por registro)
with open("datos_entrenamiento2.jsonl", "w", encoding='utf-8') as f:
    for _, row in df.iterrows():
        json_record = {
            "messages": [
                {"role": "system", "content": row["prompt"]},
                {"role": "user", "content": row["descripcion_video"]},
                {"role": "assistant", "content": row["respuesta_modelo"]}
            ]
        }
        f.write(json.dumps(json_record, ensure_ascii=False) + "\n")


In [16]:
from openai import OpenAI

client = OpenAI()

# Subir archivo de entrenamiento a OpenAI
file = client.files.create(
    file=open("datos_entrenamiento2.jsonl", "rb"),
    purpose='fine-tune'
)

# Guarda el ID generado del archivo
print(file.id)

file-U7sfBYoxkEgocb7HNqRuF2


In [13]:
files = client.files.list()
for file in files.data:
    print(file.id, file.filename, file.purpose, file.created_at)


file-Ck9urN3fqBKSTd8nffYY9z datos_entrenamiento.jsonl fine-tune 1741728664
file-4Xssq6ykry2sTbyF19d82P income_distribution.png vision 1738633138


In [17]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(
    training_file="file-U7sfBYoxkEgocb7HNqRuF2",
    model="ft:gpt-4o-mini-2024-07-18:personal::BA1zm53p"
)

FineTuningJob(id='ftjob-ZcSZQWOCbi296pXYwsi4MDT3', created_at=1741789638, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='ft:gpt-4o-mini-2024-07-18:personal::BA1zm53p', object='fine_tuning.job', organization_id='org-oUWfGFqt2HEUlKrw4qO07UUc', result_files=[], seed=1457717538, status='validating_files', trained_tokens=None, training_file='file-U7sfBYoxkEgocb7HNqRuF2', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None, metadata=None)

In [14]:
job = client.fine_tuning.jobs.retrieve("ftjob-8VgSVbMHiVeDVIcXDFTufbRr")
print(job.status)

running


In [8]:
print(job.error)


Error(code=None, message=None, param=None)
